# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

In [3]:
len(CORD19_CSV['doi'])

77448

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

In [5]:
#delete Nans
doi_counted = doi.value_counts()
doi_counted

10.31729/jnma.5498              2
10.1016/j.dsx.2020.04.012       2
10.1007/s11250-020-02439-8      1
10.1016/j.jiph.2020.06.028      1
10.1098/rsif.2018.0220          1
                               ..
10.1136/bmjopen-2017-020941     1
10.1097/gox.0000000000003007    1
10.1007/s10461-020-02891-5      1
10.1016/j.jinf.2020.11.023      1
10.4168/aair.2013.5.4.216       1
Name: doi, Length: 74302, dtype: int64

In [6]:
def AffiliationsFromScopusByDOI(client, doi):
    """obtain addiotional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

In [7]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()
## Initialize client
client = ElsClient(config['apikey'])
# https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

In [8]:
df_current_extra_info = pd.read_pickle('extra_info_CS5099.pkl')
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
23506,"[{'affiliation-city': 'Barcelona', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85092921307', ..."
23507,"[{'affiliation-city': 'Guangzhou', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '3',..."
23508,None,None
23509,"[{'affiliation-city': 'Tlalpan', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85097790720', ..."


In [9]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

23511

In [ ]:
%%time
i = len_df_current_extra_info
dict_new_extra_info = dict()
len_dois = len(doi_counted)
while i < len_dois:
    print("Position: " + str(i) + " -> " +  doi_counted.index[i])
    dict_new_extra_info[i] = AffiliationsFromScopusByDOI(client, doi_counted.index[i])
    i = i + 1 

Position: 23511 -> 10.1016/j.giq.2010.04.003
Position: 23512 -> 10.1007/s10140-020-01819-9
Position: 23513 -> 10.1016/s0168-7069(06)16004-8
Position: 23514 -> 10.1016/j.heliyon.2020.e04865
Position: 23515 -> 10.1016/j.drugpo.2011.08.008
Position: 23516 -> 10.1016/j.techfore.2020.120447
Position: 23517 -> 10.1007/s12026-020-09154-4
Position: 23518 -> 10.1093/heapro/daaa113
Position: 23519 -> 10.1016/j.virol.2008.09.032
Position: 23520 -> 10.1016/j.virol.2019.01.009
Position: 23521 -> 10.14744/anatoljcardiol.2020.57431
Position: 23522 -> 10.1186/s12974-020-01977-0
Position: 23523 -> 10.1101/2020.12.15.20248252
Position: 23524 -> 10.1128/aac.01168-20
Position: 23525 -> 10.1007/s11869-010-0079-2
Position: 23526 -> 10.1017/s1463423620000304
Position: 23527 -> 10.1101/835223
Position: 23528 -> 10.1093/jalm/jfaa089
Position: 23529 -> 10.1016/j.frl.2020.101682
Position: 23530 -> 10.1016/j.scitotenv.2020.139960
Position: 23531 -> 10.3389/fmed.2020.00542
Position: 23532 -> 10.1038/srep37725
Posi

Position: 23692 -> 10.1016/j.bja.2020.11.036
Position: 23693 -> 10.1016/j.procs.2020.09.289
Position: 23694 -> 10.1371/journal.pone.0135940
Position: 23695 -> 10.1093/jb/mvw032
Position: 23696 -> 10.1371/journal.pone.0060345
Position: 23697 -> 10.1111/tesg.12438
Position: 23698 -> 10.1007/s13042-020-01155-x
Position: 23699 -> 10.1097/pq9.0000000000000267
Position: 23700 -> 10.1016/j.prevetmed.2015.10.020
Position: 23701 -> 10.1007/s42399-020-00613-5
Position: 23702 -> 10.1038/s41598-017-05092-8
Position: 23703 -> 10.1371/journal.pcbi.1002993
Position: 23704 -> 10.1093/bioinformatics/btaa1002
Position: 23705 -> 10.1007/s41062-020-00326-7
Position: 23706 -> 10.1186/s12878-016-0069-1
Position: 23707 -> 10.3389/fimmu.2020.02130
Position: 23708 -> 10.1101/2020.11.16.20232413
Position: 23709 -> 10.2196/21995
Position: 23710 -> 10.1007/s00330-020-07271-0
Position: 23711 -> 10.3390/vaccines8030472
Position: 23712 -> 10.1007/s10742-020-00233-5
Position: 23713 -> 10.1586/14760584.2015.955478
Pos

Position: 23872 -> 10.1007/s13238-013-3096-8
Position: 23873 -> 10.1128/mbio.00686-20
Position: 23874 -> 10.1038/s41531-020-00128-9
Position: 23875 -> 10.1101/2020.07.10.20147777
Position: 23876 -> 10.1186/s12873-021-00402-w
Position: 23877 -> 10.2527/jas.2016.1000
Position: 23878 -> 10.1093/biolre/ioaa060
Position: 23879 -> 10.3345/kjp.2018.06989
Position: 23880 -> 10.1016/j.cpem.2016.05.005
Position: 23881 -> 10.3390/ijms17101743
Position: 23882 -> 10.1007/s11373-008-9279-2
Position: 23883 -> 10.1016/j.esxm.2020.100295
Position: 23884 -> 10.1016/j.actatropica.2018.12.016
Position: 23885 -> 10.1007/s10943-020-01122-4
Position: 23886 -> 10.1016/j.resconrec.2020.105052
Position: 23887 -> 10.1016/j.jaci.2016.04.063
Position: 23888 -> 10.1016/j.bja.2020.07.016
Position: 23889 -> 10.7759/cureus.9201
Position: 23890 -> 10.1016/j.chaos.2020.110051
Position: 23891 -> 10.1007/s00146-020-00988-y
Position: 23892 -> 10.2147/ijn.s269880
Position: 23893 -> 10.12688/wellcomeopenres.15933.1
Position:

Position: 24055 -> 10.1056/nejmoa2033130
Position: 24056 -> 10.1016/j.epidem.2018.04.001
Position: 24057 -> 10.1101/2020.04.23.056838
Position: 24058 -> 10.1007/s00134-020-06229-6
Position: 24059 -> 10.1111/dom.14134
Position: 24060 -> 10.1101/2020.04.06.027318
Position: 24061 -> 10.1007/s10875-020-00925-0
Position: 24062 -> 10.1186/s12889-020-10085-w
Position: 24063 -> 10.1186/gm119
Position: 24064 -> 10.1016/j.antiviral.2017.11.001
Position: 24065 -> 10.1007/s42496-020-00054-2
Position: 24066 -> 10.1007/978-3-030-50371-0_17
Position: 24067 -> 10.1007/s00264-020-04575-0
Position: 24068 -> 10.1371/journal.pone.0226952
Position: 24069 -> 10.1016/j.jff.2017.11.038
Position: 24070 -> 10.1016/j.molcel.2020.11.025
Position: 24071 -> 10.3390/ijerph17238761
Position: 24072 -> 10.1016/j.bbmt.2018.12.005
Position: 24073 -> 10.3390/ijerph17249520
Position: 24074 -> 10.1111/j.1745-7254.2008.00808.x
Position: 24075 -> 10.1016/j.rmed.2010.03.028
Position: 24076 -> 10.7150/ntno.20301
Position: 24077

Position: 24238 -> 10.1186/s12872-020-01785-7
Position: 24239 -> 10.1371/journal.pone.0141545
Position: 24240 -> 10.1007/s42979-020-00424-4
Position: 24241 -> 10.1101/2020.09.08.286732
Position: 24242 -> 10.1016/j.str.2018.09.006
Position: 24243 -> 10.1644/12-mamm-a-224.1
Position: 24244 -> 10.3390/v12060589
Position: 24245 -> 10.1002/emp2.12080
Position: 24246 -> 10.1186/s40814-020-00702-1
Position: 24247 -> 10.1016/b978-141600119-5.50016-0
Position: 24248 -> 10.1111/1755-0998.12891
Position: 24249 -> 10.1016/j.tourman.2020.104080
Position: 24250 -> 10.1093/geronb/gbaa119
Position: 24251 -> 10.1111/vru.12459
Position: 24252 -> 10.1186/s12992-020-00602-2
Position: 24253 -> 10.1101/2020.04.17.20069641
Position: 24254 -> 10.1007/s11277-021-08104-z
Position: 24255 -> 10.1101/837641
Position: 24256 -> 10.1016/j.jhin.2015.02.004
Position: 24257 -> 10.1111/anae.15188
Position: 24258 -> 10.1016/j.techfore.2012.12.008
Position: 24259 -> 10.1021/acsanm.0c01978
Position: 24260 -> 10.7554/elife.5

...

Position: 34606 -> 10.1016/j.bjid.2020.07.010
Position: 34607 -> 10.1002/art.41409
Position: 34608 -> 10.1016/j.otc.2020.09.024
Position: 34609 -> 10.1371/journal.pone.0030346
Position: 34610 -> 10.1016/j.mayocp.2020.04.028
Position: 34611 -> 10.1038/s41433-019-0762-y
Position: 34612 -> 10.3390/v12050576
Position: 34613 -> 10.1007/s11606-020-05970-4
Position: 34614 -> 10.21203/rs.3.rs-100914/v1
Position: 34615 -> 10.1136/bmjoq-2019-000895
Position: 34616 -> 10.1128/mra.00959-19
Position: 34617 -> 10.1186/s12879-020-05580-9
Position: 34618 -> 10.5811/westjem.2020.7.48436
Position: 34619 -> 10.1016/j.vetmic.2007.03.005
Position: 34620 -> 10.1111/apa.15515
Position: 34621 -> 10.1371/journal.pone.0151677
Position: 34622 -> 10.1007/s13205-020-02581-y
Position: 34623 -> 10.1186/s12889-018-5284-1
Position: 34624 -> 10.1101/2020.05.18.20099325
Position: 34625 -> 10.3233/xst-200710
Position: 34626 -> 10.1016/j.ijdrr.2020.101780
Position: 34627 -> 10.1007/s12115-020-00510-6
Position: 34628 -> 10

Position: 34787 -> 10.1007/s41999-020-00389-w
Position: 34788 -> 10.1016/j.antiviral.2021.105015
Position: 34789 -> 10.1002/jmv.26329
Position: 34790 -> 10.1016/j.vaccine.2013.05.091
Position: 34791 -> 10.3390/ijerph18010311
Position: 34792 -> 10.1016/j.jfms.2004.01.003
Position: 34793 -> 10.1007/s12020-020-02564-2
Position: 34794 -> 10.1016/j.hrthm.2020.06.016
Position: 34795 -> 10.1016/j.jpainsymman.2020.07.005
Position: 34796 -> 10.1016/j.jradnu.2020.06.008
Position: 34797 -> 10.1186/s12936-020-03296-4
Position: 34798 -> 10.1101/2020.07.07.184374
Position: 34799 -> 10.1093/ve/veaa094
Position: 34800 -> 10.1016/s0016-5085(04)80014-3
Position: 34801 -> 10.1371/journal.pbio.3000697
Position: 34802 -> 10.1016/j.mimet.2012.01.001
Position: 34803 -> 10.3389/fimmu.2019.01231
Position: 34804 -> 10.1093/intqhc/mzy033
Position: 34805 -> 10.1007/s00420-020-01543-4
Position: 34806 -> 10.1101/2020.09.04.20188417
Position: 34807 -> 10.1371/journal.pone.0244453
Position: 34808 -> 10.3389/fpsyg.202

Position: 34971 -> 10.1128/mbio.02451-20
Position: 34972 -> 10.18063/ijb.v6i4.278
Position: 34973 -> 10.1016/j.nbd.2020.105131
Position: 34974 -> 10.1002/jmv.26417
Position: 34975 -> 10.1101/2020.06.25.172312
Position: 34976 -> 10.1080/07391102.2020.1761882
Position: 34977 -> 10.1016/j.echo.2020.08.014
Position: 34978 -> 10.1017/dmp.2020.101
Position: 34979 -> 10.1016/j.bbamem.2010.08.004
Position: 34980 -> 10.1016/j.cca.2020.06.040
Position: 34981 -> 10.1007/s00134-018-5269-7
Position: 34982 -> 10.1111/j.0300-9475.2004.01423am.x
Position: 34983 -> 10.1007/s11423-020-09871-5
Position: 34984 -> 10.1016/j.ejmech.2020.112653
Position: 34985 -> 10.1007/s00104-010-1894-6
Position: 34986 -> 10.1101/2020.03.27.20045328
Position: 34987 -> 10.1016/j.aucc.2020.06.004
Position: 34988 -> 10.1007/7355_2008_026
Position: 34989 -> 10.1016/j.trc.2020.102667
Position: 34990 -> 10.1155/2017/1359064
Position: 34991 -> 10.1016/s0022-2836(03)00784-8
Position: 34992 -> 10.1007/s15010-020-01538-w
Position: 3

Position: 35153 -> 10.1007/s00277-009-0895-3
Position: 35154 -> 10.1038/s41401-019-0250-8
Position: 35155 -> 10.1038/ncomms12777
Position: 35156 -> 10.1186/s40814-019-0464-z
Position: 35157 -> 10.1016/j.celrep.2020.107720
Position: 35158 -> 10.1016/j.ijid.2020.06.026
Position: 35159 -> 10.1007/978-3-030-52240-7_28
Position: 35160 -> 10.1101/2020.04.02.20051052
Position: 35161 -> 10.1016/j.hlc.2020.10.025
Position: 35162 -> 10.1016/j.virusres.2017.04.008
Position: 35163 -> 10.1128/jcm.00375-20
Position: 35164 -> 10.1371/journal.pone.0142020
Position: 35165 -> 10.1016/j.tracli.2020.10.004
Position: 35166 -> 10.1101/2020.06.14.20130732
Position: 35167 -> 10.1007/s13311-020-00920-8
Position: 35168 -> 10.1101/2020.12.27.424507
Position: 35169 -> 10.1016/j.jcv.2020.104384
Position: 35170 -> 10.1128/mbio.01422-18
Position: 35171 -> 10.1038/nm1024
Position: 35172 -> 10.3390/ijerph17238839
Position: 35173 -> 10.3389/fpsyt.2020.605140
Position: 35174 -> 10.1186/s40413-016-0096-1
Position: 35175 

Position: 35336 -> 10.1016/b978-0-12-818706-7.00007-3
Position: 35337 -> 10.1016/j.transci.2007.09.004
Position: 35338 -> 10.1007/s00134-013-3135-1
Position: 35339 -> 10.1038/ncomms9749
Position: 35340 -> 10.1099/acmi.0.000120
Position: 35341 -> 10.1080/21645515.2015.1021527
Position: 35342 -> 10.1016/j.heliyon.2020.e04965
Position: 35343 -> 10.1111/1541-4337.12358
Position: 35344 -> 10.1007/s00438-020-01684-8
Position: 35345 -> 10.1007/s11262-011-0635-3
Position: 35346 -> 10.1101/2020.10.30.20223024
Position: 35347 -> 10.3201/eid1005.030640
Position: 35348 -> 10.3390/ijerph17217722


In [ ]:
dict_new_extra_info

In [ ]:
df_new_extra_info = pd.DataFrame(dict_new_extra_info)

In [ ]:
df_new_extra_info

In [ ]:
df_new_extra_transposed = df_new_extra_info.T
df_new_extra_transposed

In [ ]:
df_combined_extra_info = pd.concat([df_current_extra_info, df_new_extra_transposed],ignore_index=True)

In [ ]:
df_combined_extra_info.to_pickle('extra_info_CS5099.pkl')

In [ ]:
entry = AffiliationsFromScopusByDOI(client, '10.1086/605034')
entry

In [ ]:
print (json.dumps(entry, ensure_ascii=False, indent=4))

In [ ]:
df_test = pd.DataFrame.from_dict(entry['affiliation'], orient='columns')
df_test

In [ ]:
entry['coredata']

In [ ]:
## ScienceDirect (full-text) document example using DOI
doi_doc = FullDoc(doi = '10.1016/S1525-1578(10)60571-5')
if doi_doc.read(client):
    print ("doi_doc.title: ", doi_doc.title)
    doi_doc.write()   
else:
    print ("Read document failed.")
doi_doc